In [449]:

import numpy as np
import pandas as pd 
import decimal


In [450]:
def read_excel(path,sheet_name):
    df = pd.read_excel(io = path,sheet_name =sheet_name,  header = 0,index_col=None,)

   
    # name_column = df.columns.tolist()
    # for s in range(len(name_column)):
    # # del space start
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.lstrip()
    # # del space end
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.rstrip()
    return df

 
def to_json(df, nameFile):
    df.to_json(path_or_buf ='../json/m/'+nameFile+'.json' , orient="records") 
    


def get_patch_from(file):
    return '../excel/'+file+'.xlsx'

def change_types(df,columnType):
    for k, v in columnType.items(): 
        df[k] = df[k].astype(v)


def to_list_float(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [float(x) for x in df[nameCol].values[index].split(',')]

def to_list_String(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [str(x) for x in df[nameCol].values[index].split(',')]
            
            
def normalize_fraction(d):
    normalized = d.normalize()
    sign, digits, exponent = normalized.as_tuple()
    if exponent > 0:
        return decimal.Decimal((sign, digits + (0,) * exponent, 0))
    else:
        return normalized
         

# m_thread

In [451]:
file_excel = 'm_thread'

sheet_names = [
    'm_diam',
    'm_profile',
    'm_tolerance',
 
               ]

# ******************************
patchFrom = get_patch_from(file_excel)


df_base = read_excel(patchFrom, sheet_names[0])





In [452]:
# column
diam = 'diam'
diamFormat = 'diamFormat'
coarsePitch = 'coarsePitch'
finePitch = 'finePitch'
extraFinePitch = 'extraFinePitch'
rangeMain = 'rangeMain'
rangeSub = 'rangeSub'

# create df from df base

In [453]:
df_update = df_base[diam].unique()
df_update = pd.DataFrame( df_update, columns=[diam])

In [454]:
diam = 0
pitch = 0
for i, row in df_update.iterrows():
    diam = df_update.diam[i]
    s = str(diam)
    
    
    df_update.at[i, diamFormat]='M '+s.rstrip('0').rstrip('.') if '.' in s else s
    
    find_row =df_base.query("diam == @diam")
    if (find_row.count().sum())>0:
        range_main =  find_row.iloc[0]['range_main']
        range_sub =  find_row.iloc[0]['range_sub']
        
        df_update.at[i, rangeMain]=range_main
        df_update.at[i, rangeSub]=range_sub
    
    
    # coarse pitch
    find_row = df_base.query("diam == @diam and type_pitch == 1")
    if (find_row.count().sum())>0:
        pitch =  find_row.iloc[0]['pitch']
        df_update.at[i, coarsePitch]=pitch
   
    # fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 2")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, finePitch]=str(pitch).replace('[','').replace(']','')  
    
    # extra fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 3")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, extraFinePitch]=str(pitch).replace('[','').replace(']','')  




# записываем диаметры

In [455]:
to_list_float(df_update,finePitch)
to_list_float(df_update,extraFinePitch)
to_json(df_update,'m_diams')
    

In [456]:
coarsePitch = 'coarsePitch'

In [457]:
# df_pitch = pd.DataFrame(columns=[diamPitch])
diamFormat = 0
for i, row in df_update.iterrows():
    df_pitch = pd.DataFrame()
    diamFormat = df_update.diamFormat[i]
    coarse = df_update.coarsePitch[i]
    if not  np.isnan(coarse):
        df_pitch.at[i, coarsePitch]=diamFormat +' x ' + str(coarse)
        to_json(df_pitch, diamFormat.replace(' ','-'))
    
        